# 04. Regularization (정규화)

## 학습 목표
- Overfitting의 원인 이해
- L1, L2 Regularization 비교
- Dropout 원리 이해 및 구현
- Early Stopping 개념

## 관련 면접 질문
- L1과 L2 regularization의 차이는?
- Dropout이 왜 효과적인가요?
- L1이 sparse solution을 만드는 이유는?

---
## Pre-Quiz

### Q1. Overfitting이란?
```
당신의 답: 
```

### Q2. L1 vs L2 정규화의 차이점을 설명하세요
```
당신의 답: 
```

### Q3. Dropout rate가 0.5라는 것은 무슨 의미인가요?
```
당신의 답: 
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

np.random.seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

---
## 1. Overfitting 시각화

In [ ]:
# 데이터 생성: y = sin(x) + noise
n_samples = 30
X = np.sort(np.random.rand(n_samples) * 4 * np.pi)
y = np.sin(X) + np.random.randn(n_samples) * 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 다양한 복잡도의 모델 비교
degrees = [1, 4, 15]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

X_plot = np.linspace(0, 4*np.pi, 100)

for ax, degree in zip(axes, degrees):
    poly = PolynomialFeatures(degree)
    X_train_poly = poly.fit_transform(X_train.reshape(-1, 1))
    X_plot_poly = poly.transform(X_plot.reshape(-1, 1))
    
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    y_plot = model.predict(X_plot_poly)
    
    ax.scatter(X_train, y_train, c='blue', label='Train')
    ax.scatter(X_test, y_test, c='red', label='Test')
    ax.plot(X_plot, y_plot, 'g-', lw=2)
    ax.plot(X_plot, np.sin(X_plot), 'k--', alpha=0.5, label='True')
    ax.set_title(f'Degree {degree}')
    ax.legend()
    ax.set_ylim(-2, 2)

plt.suptitle('Underfitting → Good Fit → Overfitting')
plt.tight_layout()
plt.show()

---
## 2. L1 vs L2 Regularization

### 수식

| 종류 | Loss Function | 특징 |
|------|--------------|------|
| L2 (Ridge) | $L + \lambda \sum w_i^2$ | 모든 가중치를 작게 |
| L1 (Lasso) | $L + \lambda \sum |w_i|$ | 일부 가중치를 0으로 (Sparse) |

### L1이 Sparse Solution을 만드는 이유
- L1 gradient는 상수 (±1)
- 작은 가중치도 일정한 속도로 0으로 이동
- L2 gradient는 가중치에 비례 (2w)
- 작은 가중치는 느리게 0에 접근 (완전히 0이 되지 않음)

In [ ]:
# L1 vs L2 영향 시각화
def visualize_l1_l2():
    w = np.linspace(-3, 3, 100)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. Penalty 함수
    axes[0].plot(w, np.abs(w), label='L1: |w|', lw=2)
    axes[0].plot(w, w**2, label='L2: w²', lw=2)
    axes[0].set_xlabel('w')
    axes[0].set_ylabel('Penalty')
    axes[0].set_title('Penalty 함수')
    axes[0].legend()
    axes[0].grid(True)
    
    # 2. Gradient
    axes[1].plot(w, np.sign(w), label='L1 gradient: sign(w)', lw=2)
    axes[1].plot(w, 2*w, label='L2 gradient: 2w', lw=2)
    axes[1].set_xlabel('w')
    axes[1].set_ylabel('Gradient')
    axes[1].set_title('Gradient (w에 대한 미분)')
    axes[1].legend()
    axes[1].grid(True)
    
    # 3. Contour (2D)
    w1, w2 = np.meshgrid(np.linspace(-2, 2, 100), np.linspace(-2, 2, 100))
    l1 = np.abs(w1) + np.abs(w2)
    l2 = w1**2 + w2**2
    
    axes[2].contour(w1, w2, l1, levels=[0.5, 1, 1.5, 2], colors='blue', linestyles='--')
    axes[2].contour(w1, w2, l2, levels=[0.5, 1, 1.5, 2], colors='red')
    axes[2].set_xlabel('w1')
    axes[2].set_ylabel('w2')
    axes[2].set_title('등고선: L1(파랑), L2(빨강)')
    axes[2].set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

visualize_l1_l2()

In [ ]:
# =================================================
# TODO: L1, L2 정규화 손실 함수 구현
# =================================================

def l1_penalty(weights, lambda_):
    """
    L1 정규화 penalty
    
    Args:
        weights: 가중치 배열
        lambda_: 정규화 강도
    Returns:
        penalty: lambda * sum(|w|)
    """
    # ============================================
    # 여기에 구현하세요
    # ============================================
    pass


def l2_penalty(weights, lambda_):
    """
    L2 정규화 penalty
    
    Args:
        weights: 가중치 배열
        lambda_: 정규화 강도
    Returns:
        penalty: lambda * sum(w^2)
    """
    # ============================================
    # 여기에 구현하세요
    # ============================================
    pass


def l1_gradient(weights, lambda_):
    """
    L1 정규화 gradient
    
    Returns:
        gradient: lambda * sign(w)
    """
    # ============================================
    # 여기에 구현하세요
    # ============================================
    pass


def l2_gradient(weights, lambda_):
    """
    L2 정규화 gradient
    
    Returns:
        gradient: 2 * lambda * w
    """
    # ============================================
    # 여기에 구현하세요
    # ============================================
    pass

In [ ]:
# ============ 테스트 ============
def test_regularization():
    w = np.array([1.0, -2.0, 0.5])
    lambda_ = 0.1
    
    # L1
    assert np.isclose(l1_penalty(w, lambda_), 0.35), "L1 penalty 오류"
    expected_l1_grad = np.array([0.1, -0.1, 0.1])
    assert np.allclose(l1_gradient(w, lambda_), expected_l1_grad), "L1 gradient 오류"
    
    # L2
    assert np.isclose(l2_penalty(w, lambda_), 0.525), "L2 penalty 오류"
    expected_l2_grad = np.array([0.2, -0.4, 0.1])
    assert np.allclose(l2_gradient(w, lambda_), expected_l2_grad), "L2 gradient 오류"
    
    print("✅ Regularization 테스트 통과!")

test_regularization()

---
## 3. Ridge vs Lasso 비교

In [ ]:
# 고차원 데이터에서 Ridge vs Lasso
np.random.seed(42)

# 100개 feature 중 10개만 실제로 유의미
n_features = 100
n_informative = 10
n_samples = 50

# 실제 가중치 (10개만 non-zero)
true_weights = np.zeros(n_features)
true_weights[:n_informative] = np.random.randn(n_informative) * 5

X = np.random.randn(n_samples, n_features)
y = X @ true_weights + np.random.randn(n_samples) * 0.5

# 모델 학습
alpha = 1.0
ridge = Ridge(alpha=alpha).fit(X, y)
lasso = Lasso(alpha=alpha).fit(X, y)

# 가중치 비교
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].bar(range(n_features), true_weights)
axes[0].set_title('True Weights (10개만 non-zero)')
axes[0].set_xlabel('Feature Index')

axes[1].bar(range(n_features), ridge.coef_)
axes[1].set_title('Ridge Weights (모든 feature 사용)')
axes[1].set_xlabel('Feature Index')

axes[2].bar(range(n_features), lasso.coef_)
axes[2].set_title(f'Lasso Weights (non-zero: {np.sum(lasso.coef_ != 0)}개)')
axes[2].set_xlabel('Feature Index')

plt.tight_layout()
plt.show()

print(f"Ridge non-zero weights: {np.sum(ridge.coef_ != 0)}")
print(f"Lasso non-zero weights: {np.sum(lasso.coef_ != 0)}")

---
## 4. Dropout 구현

### 원리
- **학습 시**: 각 뉴런을 확률 p로 비활성화 (출력을 0으로)
- **추론 시**: 모든 뉴런 활성화, 출력에 (1-p) 곱함
- 또는 **Inverted Dropout**: 학습 시 1/(1-p) 곱함, 추론 시 변경 없음

### 효과
- 앙상블 효과: 매번 다른 서브네트워크 학습
- Co-adaptation 방지: 특정 뉴런에 의존하지 않도록

In [ ]:
# =================================================
# TODO: Dropout 레이어 구현 (Inverted Dropout)
# =================================================

class Dropout:
    def __init__(self, p=0.5):
        """
        Args:
            p: dropout 확률 (비활성화할 비율)
        """
        self.p = p
        self.mask = None
        self.training = True
    
    def forward(self, x):
        """
        Args:
            x: input array
        Returns:
            out: dropout 적용된 output
        """
        # ============================================
        # 여기에 구현하세요
        # training=True:
        #   1. mask 생성: np.random.rand(*x.shape) > p
        #   2. x * mask / (1 - p)  (inverted dropout)
        # training=False:
        #   그대로 반환
        # ============================================
        pass
    
    def backward(self, dout):
        """
        Args:
            dout: upstream gradient
        Returns:
            dx: gradient (mask 적용)
        """
        # ============================================
        # 여기에 구현하세요
        # mask된 위치의 gradient는 0
        # ============================================
        pass
    
    def train(self):
        self.training = True
    
    def eval(self):
        self.training = False

In [ ]:
# ============ 테스트 ============
def test_dropout():
    np.random.seed(42)
    
    dropout = Dropout(p=0.5)
    x = np.ones((100, 100))
    
    # Training mode
    dropout.train()
    out = dropout.forward(x)
    
    # 약 50%가 0이어야 함
    zero_ratio = np.mean(out == 0)
    assert 0.4 < zero_ratio < 0.6, f"Dropout 비율 오류: {zero_ratio}"
    
    # Non-zero 값은 2배 (1 / (1-0.5))
    non_zero_vals = out[out != 0]
    assert np.allclose(non_zero_vals, 2.0), f"Inverted scaling 오류: {non_zero_vals.mean()}"
    
    # Eval mode - 변경 없어야 함
    dropout.eval()
    out_eval = dropout.forward(x)
    assert np.allclose(out_eval, x), "Eval mode에서는 변경 없어야 함"
    
    print("✅ Dropout 테스트 통과!")

test_dropout()

---
## 5. Dropout 효과 시각화

In [ ]:
def visualize_dropout():
    """Dropout의 마스킹 효과 시각화"""
    np.random.seed(42)
    
    x = np.random.randn(8, 8)
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    dropout_rates = [0, 0.3, 0.5, 0.7]
    
    for ax, p in zip(axes, dropout_rates):
        np.random.seed(42)
        dropout = Dropout(p=p)
        out = dropout.forward(x)
        
        im = ax.imshow(out, cmap='RdBu', vmin=-3, vmax=3)
        ax.set_title(f'Dropout p={p}')
        ax.axis('off')
    
    plt.colorbar(im, ax=axes, shrink=0.8)
    plt.suptitle('Dropout: 검은색(0) = 비활성화된 뉴런')
    plt.show()

visualize_dropout()

---
## 6. Early Stopping

### 개념
- Validation loss가 개선되지 않으면 학습 조기 종료
- Overfitting 방지의 간단하면서 효과적인 방법

In [ ]:
# =================================================
# TODO: Early Stopping 구현
# =================================================

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Args:
            patience: 개선 없이 기다릴 epoch 수
            min_delta: 개선으로 인정할 최소 변화량
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.should_stop = False
    
    def __call__(self, val_loss):
        """
        Args:
            val_loss: 현재 validation loss
        Returns:
            should_stop: True면 학습 중단
        """
        # ============================================
        # 여기에 구현하세요
        # 1. best_loss가 None이면 초기화
        # 2. val_loss < best_loss - min_delta 이면 개선
        #    - best_loss 업데이트, counter 리셋
        # 3. 아니면 counter += 1
        # 4. counter >= patience 이면 should_stop = True
        # ============================================
        pass

In [ ]:
# Early Stopping 시뮬레이션
np.random.seed(42)

# 가상의 loss 곡선 (train은 계속 감소, val은 나중에 증가)
epochs = 100
train_loss = 1.0 * np.exp(-np.arange(epochs) / 20)
val_loss = train_loss + 0.1 * np.exp((np.arange(epochs) - 50) / 20)
val_loss[:50] = train_loss[:50] + np.random.randn(50) * 0.02

# Early stopping 적용
early_stop = EarlyStopping(patience=10)
stop_epoch = epochs

for epoch in range(epochs):
    if early_stop(val_loss[epoch]):
        stop_epoch = epoch
        break

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Val Loss')
plt.axvline(x=stop_epoch, color='r', linestyle='--', label=f'Early Stop (epoch {stop_epoch})')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Early Stopping 효과')
plt.legend()
plt.grid(True)
plt.show()

print(f"Early stopping at epoch {stop_epoch}")

---
## Post-Quiz

### Q1. L1 정규화가 feature selection에 유용한 이유는?
```
당신의 답: 
```

### Q2. Dropout rate를 너무 높게 설정하면 어떤 문제가 발생하나요?
```
당신의 답: 
```

### Q3. Batch Normalization이 dropout의 대안이 될 수 있는 이유는?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
def l1_penalty(weights, lambda_):
    return lambda_ * np.sum(np.abs(weights))

def l2_penalty(weights, lambda_):
    return lambda_ * np.sum(weights ** 2)

def l1_gradient(weights, lambda_):
    return lambda_ * np.sign(weights)

def l2_gradient(weights, lambda_):
    return 2 * lambda_ * weights


class Dropout:
    def __init__(self, p=0.5):
        self.p = p
        self.mask = None
        self.training = True
    
    def forward(self, x):
        if self.training:
            self.mask = (np.random.rand(*x.shape) > self.p).astype(np.float64)
            return x * self.mask / (1 - self.p)
        else:
            return x
    
    def backward(self, dout):
        return dout * self.mask / (1 - self.p)
    
    def train(self):
        self.training = True
    
    def eval(self):
        self.training = False


class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.should_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        return self.should_stop
```

</details>

<details>
<summary>👉 Quiz 정답</summary>

### Pre-Quiz

**Q1. Overfitting이란?**
모델이 학습 데이터에 과도하게 적합되어 새로운 데이터에 대한 일반화 성능이 떨어지는 현상. Train loss는 낮지만 Val/Test loss가 높음.

**Q2. L1 vs L2 차이**
- L1: 가중치 절대값 합에 페널티 → 일부 가중치를 정확히 0으로 (sparse)
- L2: 가중치 제곱 합에 페널티 → 모든 가중치를 작게 (0에 가깝게)

**Q3. Dropout rate 0.5**
학습 중 각 뉴런이 50% 확률로 비활성화(출력 0). 매 iteration마다 다른 서브네트워크로 학습하는 효과.

### Post-Quiz

**Q1. L1이 feature selection에 유용한 이유**
L1은 일부 가중치를 정확히 0으로 만들어 해당 feature를 제거하는 효과. 어떤 feature가 중요한지 자동 선택됨.

**Q2. Dropout rate가 너무 높으면**
너무 많은 뉴런이 비활성화되어 학습이 불안정하고 느려짐. 충분한 정보가 전달되지 않아 underfitting 발생 가능.

**Q3. BatchNorm이 dropout 대안인 이유**
BatchNorm도 정규화 효과가 있어 overfitting 방지. 미니배치의 노이즈가 암묵적 정규화 역할. 많은 현대 모델에서 dropout 없이 BatchNorm만 사용.

</details>